In [1]:
#V_and_W_diff_eqs_alt.jl


using DifferentialEquations

function timestep(dt, zeta, Delta, V_old, W_old)
    n = size(zeta, 1)
    
    V_new = (I - 1im * Delta * dt) * V_old - 2im * zeta * dt * conj(W_old)
    W_new = (I - 1im * Delta * dt) * W_old - 2im * zeta * dt * conj(V_old)
    
    return V_new, W_new
end

function odefun(dydt, y, t, Zeta, Delta)
    n = size(Zeta, 1)
    
    tpos = round(interp1(t, 1:length(t), t))
    
    V_vec = y[1:n*n]
    W_vec = y[n*n+1:2*n*n]
    
    # Turn them back into nXn matrices
    V = reshape(V_vec, n, n)
    W = reshape(W_vec, n, n)
    
    dydt_V = -1im * Delta * V - 2 * 1im * Zeta[:, :, tpos] * conj(W)
    dydt_W = -1im * Delta * W - 2 * 1im * Zeta[:, :, tpos] * conj(V)
    
    dydt[1:n*n] = dydt_V[:]
    dydt[n*n+1:2*n*n] = dydt_W[:]
end
using LinearAlgebra  # Make sure to have the LinearAlgebra package installed

function calculate_parameters(tbar, zeta, Delta, tbar_min, tbar_max, GammaP)
    n = size(zeta, 1)
    
    V0 = I
    W0 = zeros(ComplexF64, n, n)
    
    y0 = vcat(vec(V0), vec(W0))
    
    prob = ODEProblem((dydt, y, t) -> odefun(dydt, y, t, zeta / GammaP, Delta / GammaP), y0, (tbar_min, tbar_max))
    sol = solve(prob, ode45)
    
    V = zeros(ComplexF64, n, n, length(sol.t))
    W = zeros(ComplexF64, n, n, length(sol.t))
    
    squeeze_parameter = zeros(ComplexF64, n, n, length(sol.t))
    rotation_parameter = zeros(ComplexF64, n, n, length(sol.t))
    
    for (idx, t_val) in enumerate(sol.t)
        V1 = reshape(sol[idx][1:n*n], n, n)
        W1 = reshape(sol[idx][n*n+1:2*n*n], n, n)
        
        V[:, :, idx] = V1
        W[:, :, idx] = W1
        
        # Check constraints
        c1_check = maximum(abs.(V1 * W1' - W1 * V1'))
        c2_check1 = 100 * maximum(abs.(diagm(V1 * V1' - W1 * W1' - I)))
        c2_check2 = maximum(abs.(V1 * V1' - W1 * W1' - diagm(diagm(V1 * V1' - W1 * W1'))))
        
        if c1_check > 1e-6
            println("c1 bad at GammaP t = ", t_val)
            break
        elseif c2_check1 > 1
            println("diag > 1 by 1% at GammaP t by = ", t_val)
            break
        elseif c2_check2 > 1e-6
            println("c2 bad at GammaP t by = ", t_val)
            break
        end

        coshu, exp_phi = polar(V1)

        F, coshr = schur(coshu)

        coshr_diag = diagm(coshr)

        coshr_check = real(diagm(coshr - I))
        coshr_diag[coshr_check .< 1e-10] .= 1.0

        F_check = maximum(abs.(F * F' - I))
        if F_check > 1e-12
            println("F is not unitary")
        end

        r_diag = acosh.(coshr_diag)
        r = diagm(r_diag)

        u = F * r * F'
        sinhr = diagm(sinh.(r_diag))
        sinhu = F * sinhr * F'

        Pinv_eig = [real(r_diag[i]) < 1e-12 ? 1.0 : r_diag[i] / sinhr[i, i] for i in 1:length(r_diag)]
        P_eig = [real(r_diag[i]) < 1e-12 ? 1.0 : sinhr[i, i] / r_diag[i] for i in 1:length(r_diag)]

        Pinv = F * diagm(Pinv_eig) * F'
        P = F * diagm(P_eig) * F'

        J = Pinv * (W1 * exp_phi')

        W_new = P * J * conj(exp_phi)

        Wcheck = 100 * 2 * abs.(W_new - W1) ./ abs.(W_new + W1)
        Wcheck = maximum(Wcheck)
        
        println("max percentage difference between W_new and W = ", Wcheck, " %")

        squeeze_parameter[:, :, idx] = r
        rotation_parameter[:, :, idx] = exp_phi
    end
    
    return V, W, squeeze_parameter, rotation_parameter
end


calculate_parameters (generic function with 1 method)

In [2]:
#nonlinear_and_detuning_parameter.jl

using SpecialFunctions  # For the erf function

function calculate_parameters(λ1, λ2, λS, Q, τ1, τ2, L, vg, η, P1_peak, P2_peak, γ_nl, R, neff, tbar_min, tbar_max, dtbar)
    c = 3.0e8  # Speed of light in m/s

    omega1 = 2 * π * c / λ1
    omega2 = 2 * π * c / λ2
    omegaS = 2 * π * c / λS

    GammaS = omegaS / (2 * Q)
    GammaP1 = omega1 / (2 * Q)
    GammaP2 = omega2 / (2 * Q)
    GammaP = (GammaP1 + GammaP2) * 0.5

    beta1 = GammaP1 * τ1 / sqrt(8 * log(2))
    beta2 = GammaP2 * τ2 / sqrt(8 * log(2))

    z0 = -η^2 * sqrt(P1_peak * P2_peak) * vg^3 * τ1 * τ2 * γ_nl / (4 * log(2) * L * 2 * π * R)

    dl = 0.05e-9
    kS_max = 2 * π * neff / (λS - dl)
    kS_min = 2 * π * neff / (λS + dl)
    dk = 2 * π / L
    k1 = kS_min:dk:kS_max
    k2 = k1
    KS = 2 * π * neff / λS

    zeta_x = zeros(ComplexF64, length(k1), length(k2))

    for (idx1, k1_val) in enumerate(k1)
        x1 = vg * (k1_val - KS) / GammaS

        for (idx2, k2_val) in enumerate(k2)
            x2 = vg * (k2_val - KS) / GammaS
            zeta_x[idx1, idx2] = 1 / (1im * x1 + 1) * 1 / (1im * x2 + 1)
        end
    end

    tbar = tbar_min:dtbar:tbar_max

    zeta1_t = zeros(ComplexF64, length(tbar))
    zeta2_t = zeros(ComplexF64, length(tbar))

    for (idx1, tbar_val) in enumerate(tbar)
        a1 = 1 / (2 * beta1)
        println(typeof(tbar_val))
        println(typeof(a1))
        println(typeof(beta1))
        z1 = beta1 - a1 * tbar_val
        zeta1_t[idx1] = exp(-(a1 * tbar_val)^2) * exp(z1^2) * (1 - erf(z1))

        a2 = 1 / (2 * beta2)
        println(typeof(tbar_val))
        println(typeof(a2))
        println(typeof(beta2))
        z2 = beta2 - a2 * tbar_val
        zeta2_t[idx1] = exp(-(a2 * tbar_val)^2) * exp(z2^2) * (1 - erf(z2))
    end

    zeta = zeros(ComplexF64, length(k1), length(k2), length(tbar))

    for (idx1, tbar_val) in enumerate(tbar)
        zeta[:, :, idx1] = z0 * zeta_x * zeta1_t[idx1] * zeta2_t[idx1]
    end

    Delta = 1 * diagm(vg * (k1 - KS))
    
    T = L / vg + Q / omegaS

    return zeta, Delta, T, tbar
end


calculate_parameters (generic function with 2 methods)

In [3]:
#squeezing_and_rotation_parameters.jl

using SpecialFunctions, LinearAlgebra, DifferentialEquations, PyPlot

# Define constants first
c = 299792458
hbar = 1.05457182e-34

# Physical parameters of the waveguides
gamma_nl = 1  # 1/(Wm)
eta = 0.5  # coupling efficiency to real channel
Q = 2e5  # loaded Q, assumed independent of frequency
R = 50e-6  # ring radius
neff = 1.8  # effective index
ng = 2.1  # group index
vg = c / ng
TR = 2 * π * R * neff / c  # ring round trip time

# Pump pulse energy and duration
λ1 = 1545e-9  # wavelength of pump 1
λ2 = 1555e-9  # wavelength of pump 2
wP1 = 2 * π * c / λ1
wP2 = 2 * π * c / λ2
wS = 0.5 * (wP1 + wP2)  # frequency of squeezed light
λS = 2 * π * c / wS

reprate = 70e3  # Hz
P1_peak = 3  # peak power of pump 1 (W)
P2_peak = 3  # peak power of pump 2 (W)
τ1 = 100 * TR  # duration of pump 1 units of TR
τ2 = 100 * TR  # duration of pump 2 units of TR

Pavg1 = P1_peak * reprate * τ1
Pavg2 = P2_peak * reprate * τ2

# Wavevector in channel (dimensionless parameter x)
L = 0.75  # 1.5; length of channel (m)

# Set up the dimensionless time parameter...actual time t = tbar / GammaP
tbar_min = -1  # min and max times
tbar_max = 2
dtbar = (tbar_max - tbar_min) / 5000  # step size in t

# Run the script to get the nonlinear parameter
include("nonlinear_and_detuning_parameter.jl")

# Now using things from the script we just ran...

zeta, Delta, T, tbar = calculate_parameters(λ1, λ2, λS, Q, τ1, τ2, L, vg, eta, P1_peak, P2_peak, gamma_nl, R, neff, tbar_min, tbar_max, dtbar)


# This ratio should be much larger than 1
println("T/tau = ", T / τ1)

# Number of modes
n = size(zeta, 1)

# Plot maximum of zeta
k0 = argmin(abs.(k1 .- KS))
zeta_slice = reshape(zeta[k0, k0, :], 1, length(tbar))

plot(tbar, abs.(zeta_slice), linewidth=5)
ylabel("|ζ(0,0,t)|", "Interpreter", "latex")
xlabel("Γt", "Interpreter", "latex")
xticks(fontsize=12)
yticks(fontsize=12)
grid(true)
show()

# Now put this nonlinear parameter and detuning parameter into the V and W equations
# to get the squeezing and rotation parameters
include("V_and_W_diff_eqs_alt.jl")

num_sq = size(squeeze_parameter, 1)

figure()
for i in 1:num_sq
    now_squeeze_parameter = reshape(squeeze_parameter[i, i, :], 1, length(t_ode))
    
    if maximum(real(now_squeeze_parameter)) > 0.01
        plot(t_ode, real(now_squeeze_parameter), linestyle="-", marker=".", linewidth=1, label="\$r_{$(i)}\$")
        hold(true)
    end
end

ylabel("\$r\$", "Interpreter", "latex")
xlabel("\$\\Gamma t\$", "Interpreter", "latex")
xticks(fontsize=12)
yticks(fontsize=12)
legend(interpreter="latex", location="best")
grid(true)
show()

# Plot J
figure()
JR = real(J)
scatter(L1[:]*1e9, L2[:]*1e9, 20, JR[:], marker="o", cmap="jet", edgecolors="none")
colorbar()

xticks(fontsize=12)
yticks(fontsize=12)
title("\$Re(J)\$", "Interpreter", "latex")
xlabel("Generated wavelength", "Interpreter", "latex")
ylabel("Generated wavelength", "Interpreter", "latex")
grid(true)
show()



LoadError: MethodError: no method matching -(::StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}, ::Float64)
For element-wise subtraction, use broadcasting with dot syntax: array .- scalar

[0mClosest candidates are:
[0m  -([91m::T[39m, ::T) where T<:Union{Float16, Float32, Float64}
[0m[90m   @[39m [90mBase[39m [90m[4mfloat.jl:409[24m[39m
[0m  -([91m::ArrayPartition[39m, ::Number)
[0m[90m   @[39m [36mRecursiveArrayTools[39m [90m~/.julia/packages/RecursiveArrayTools/X30HP/src/[39m[90m[4marray_partition.jl:131[24m[39m
[0m  -([91m::UniformScaling[39m, ::Number)
[0m[90m   @[39m [32mLinearAlgebra[39m [90m/Applications/Julia-1.9.app/Contents/Resources/julia/share/julia/stdlib/v1.9/LinearAlgebra/src/[39m[90m[4muniformscaling.jl:146[24m[39m
[0m  ...
